In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"

In [ ]:
import os
os.system('pip install scikeras')

0

In [ ]:
import numpy as np
import pandas as pd
for dirname, _, filenames in os.walk('../archive/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/content/all.csv")
train = pd.read_csv("/content/train.csv")
val = pd.read_csv("/content/val.csv")
test = pd.read_csv("/content/test.csv")

In [ ]:
import glob
import time
import pandas as pd

from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
temp = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
t1 = temp.fit_transform(df['Sentence'].values.astype('U')).toarray()
vocabulary = temp.get_feature_names_out()
vectorizer = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'), vocabulary=vocabulary)

In [ ]:
train

,Sentence,Label
0,-2384'|| ( select 'vhcp' from dual where 1266 ...,1
1,hinkel,0
2,1 rlike ( select * from ( select ( sleep ( 5...,1
3,7081,0
4,It &apos;s apparent continuation Israel &apos...,0
...,...,...
26974,"Only people get old , says , smirking",0
26975,select * from users where id = 1 or \.<1 or 1...,1
26976,rali,0
26977,radley,0


In [ ]:
X_train = vectorizer.fit_transform(train['Sentence'].values.astype('U')).toarray()
X_val = vectorizer.fit_transform(val['Sentence'].values.astype('U')).toarray()
X_test = vectorizer.fit_transform(test['Sentence'].values.astype('U')).toarray()

In [ ]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
y_train = train['Label'].values
y_val = val['Label'].values
y_test = test['Label'].values

In [ ]:
y_train

array([1, 0, 1, ..., 0, 0, 0])

In [ ]:
print('X_train: ', X_train.shape)
print('X_val: ', X_val.shape)
print('X_test: ', X_test.shape)

X_train:  (26979, 10319)
X_val:  (3372, 10319)
X_test:  (3374, 10319)


In [ ]:
import joblib

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def performence(y_test, y_pred):
    a = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = \
        precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print("Accuracy : {:.4f}".format(a))
    print("Precision : {:.4f}".format(precision))
    print("Recall : {:.4f}".format(recall))
    print("F-Score : {:.4f}".format(fscore))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
y_test.shape

(3374,)

In [ ]:
performence(y_test, y_pred)

Accuracy : 0.9600
Precision : 0.9622
Recall : 0.9600
F-Score : 0.9593


In [ ]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from scikeras.wrappers import KerasClassifier

In [ ]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                206400    
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1024)              11264     
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1025      
                                                        

In [ ]:
classifier_nn = model.fit(X_train,y_train,epochs=10,verbose=True,validation_data=(X_test, y_test),batch_size=15)

Epoch 1/10
1799/1799 [==============================] - 26s 13ms/step - loss: 0.1551 - accuracy: 0.9563 - val_loss: 0.1245 - val_accuracy: 0.9630
Epoch 2/10
1799/1799 [==============================] - 19s 11ms/step - loss: 0.1018 - accuracy: 0.9773 - val_loss: 0.1403 - val_accuracy: 0.9701
Epoch 3/10
1799/1799 [==============================] - 17s 10ms/step - loss: 0.0870 - accuracy: 0.9825 - val_loss: 0.1379 - val_accuracy: 0.9680
Epoch 4/10
1799/1799 [==============================] - 16s 9ms/step - loss: 0.0801 - accuracy: 0.9832 - val_loss: 0.1334 - val_accuracy: 0.9659
Epoch 5/10
1799/1799 [==============================] - 21s 12ms/step - loss: 0.0790 - accuracy: 0.9847 - val_loss: 0.1535 - val_accuracy: 0.9665
Epoch 6/10
1799/1799 [==============================] - 22s 12ms/step - loss: 0.0811 - accuracy: 0.9837 - val_loss: 0.1475 - val_accuracy: 0.9650
Epoch 7/10
1799/1799 [==============================] - 24s 14ms/step - loss: 0.0807 - accuracy: 0.9838 - val_loss: 0.1501 - 

In [ ]:
pred=model.predict(X_test)

106/106 [==============================] - 0s 3ms/step


In [ ]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [ ]:
performence(y_test, pred)

Accuracy : 0.9582
Precision : 0.9586
Recall : 0.9582
F-Score : 0.9578


In [ ]:
import pickle

In [ ]:
pickle.dump(model ,open('model.pkl','wb'))

In [ ]:
import joblib
# Assuming 'model' is the variable holding your trained model
joblib.dump(model, 'mlp.pkl')

['mlp.pkl']

In [ ]:
import joblib
model = joblib.load('mlp.pkl')

In [ ]:

model.save('mlp21.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torchvision.models as models

mlp1 = models.alexnet(pretrained=True)a

torch.save(mlp1.state_dict(), '/content/drive/MyDrive/Colab Notebooks/SQL project/mlp1.pt')